In [ ]:
# Data Shaping
import polars as pl
import datetime as dt

# API Calls
import requests
from requests.structures import CaseInsensitiveDict

# File Handling and Export
import json
import os
import hvac
import traceback

In [ ]:
def read_secret(mount_point, path):
    ca_path = os.environ.get("SSL_CERT_FILE")
    client = hvac.Client(verify=ca_path)
    resp = client.kv.v2.read_secret(mount_point=mount_point, path=f'/{path}')
    secret = resp['data']['data']

    return secret

In [ ]:
def __create_token(config):
    try:
        # call token api url
        token_uri = f'{config["base_uri"]}/auth/oauth/token'

        # construct header
        headers = CaseInsensitiveDict()
        headers["Content-Type"] = "application/x-www-form-urlencoded"

        # construct req body
        data = CaseInsensitiveDict()
        data["grant_type"] = "password"
        data["username"] = config["api_key"]
        data["password"] = config["api_secret"]

        # request content response
        resp = requests.post(token_uri, headers=headers, data=data, auth=("public-client", "public"))
        content = resp.content.decode("utf-8")
        c_dict = json.loads(content)

        return {
            "access_token": c_dict["access_token"],
            "result": {
                "job_title": config["title"],
                "status_code": 200,
                "message": "Success"
            }
        }

    except Exception as e:
        return {
            "access_token": "error",
            "result": {
                "job_title": config["title"],
                "status_code": 500,
                "message": traceback.format_exc()
            }
        }

In [ ]:
def __api_pagination(config, url):
    try:
        # construct header
        headers = CaseInsensitiveDict()
        headers["Authorization"] = f'Bearer {config["base_uri"]}'
        headers["Content-Type"] = "application/json"

        # construct req body
        data = ''

        print(f'Request URL: {url}')

        resp = requests.get(url, headers=headers, data=data)
        content = resp.content.decode('utf-8')
        print(content)
        c_dict = json.loads(content)

        return {
            "data": c_dict,
            "result": {
                "status_code": 200,
                "task_title": config["title"],
                "message": "Success"
            }
        }

    except Exception as e:
        return {
            "result": {
                "job_title": config["title"],
                "status_code": 500,
                "message": traceback.format_exc()
            }
        }

In [ ]:
secrets = read_secret(mount_point="api",
                      path="datto_rmm/systems_example_co")

In [ ]:
def create_devices_dataframe(config):
    try:
        # Create Devices DataFrame
        request_url = f'{config["base_uri"]}/api/v2/account/devices'
        data = __api_pagination(config, request_url)
        c_dict = data["data"]
        return c_dict
    except Exception as e:
        print(e)



In [ ]:
secrets["title"] = "datto_rmm_api_devices"

In [ ]:
secrets.update(__create_token(secrets))

In [ ]:
result = create_devices_dataframe(secrets)

In [ ]:
result

In [ ]:
(secrets['access_token'])